# SPSS Modelerのフィールド作成ノードをPythonで書き換える。時系列センサーデータからの特徴量抽出


分析対象のデータは以下です。
M_CD: マシンコード
UP_TIIME: 起動時間
POWER: 電力
TEMP: 温度
ERR_CD: エラーコード

各マシンコードごとに起動時間にそって電力や温度の変化、そしてエラーがあればそれが時系列に記録されています。
今回はこのデータから以下のような特徴量を作ってみます。
<BR>① 条件付き：1時間前の電力の差
<BR>② フラグ型：電力が増減するジグザクを捉えるフラグ
<BR>③ カウント型:ジグザグを起こした累積回数
<BR>④ ステート型：ジグザグが頻繁に起きているか、起きていないかの状態
<BR>![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/26775713-1fc5-16b7-4f8a-cf2c4ea4dd52.png)


In [1]:
!wget https://raw.githubusercontent.com/hkwd/200611Modeler2Python/master/data/Cond4n_e.csv

--2020-11-18 04:50:23--  https://raw.githubusercontent.com/hkwd/200611Modeler2Python/master/data/Cond4n_e.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 128814 (126K) [text/plain]
Saving to: ‘Cond4n_e.csv.2’

100%[======================================>] 128,814     --.-K/s   in 0.003s  

2020-11-18 04:50:23 (47.1 MB/s) - ‘Cond4n_e.csv.2’ saved [128814/128814]



In [2]:
import pandas as pd
df = pd.read_csv('Cond4n_e.csv')
df = df.sort_values(by=['M_CD','UP_TIME']).reset_index(drop=True) 
df['M_CD'] = df['M_CD'].astype(str)
df['ERR_CD'] = df['ERR_CD'].astype(str)

In [3]:
df.head()

,M_CD,UP_TIME,POWER,TEMP,ERR_CD
0,104,0,1003,244,0
1,104,1,1003,244,0
2,104,2,1003,244,0
3,104,3,1003,244,0
4,104,4,1003,244,0


# ① 条件付き：1時間前の電力の差
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/dbd8325d-5a41-bd97-5b3d-2d131f51d067.png)


In [4]:
df['POWER_DIFF'] = df.groupby(['M_CD'])['POWER'].diff(1)
df.head()

,M_CD,UP_TIME,POWER,TEMP,ERR_CD,POWER_DIFF
0,104,0,1003,244,0,NaN
1,104,1,1003,244,0,0.0
2,104,2,1003,244,0,0.0
3,104,3,1003,244,0,0.0
4,104,4,1003,244,0,0.0


# ② フラグ型：電力が増減するジグザクを捉えるフラグ
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/1e245100-7fca-5add-afee-9d3052663802.png)

In [5]:
#1時間前のPOWER_DIFFの列を追加
df['PREV_POWER_DIFF'] = df.groupby(['M_CD'])['POWER_DIFF'].shift(1)
df.query('M_CD == "229"').head(100)


,M_CD,UP_TIME,POWER,TEMP,ERR_CD,POWER_DIFF,PREV_POWER_DIFF
1144,229,0,1072,259,0,NaN,NaN
1145,229,1,1072,259,0,0.0,NaN
1146,229,2,1072,259,0,0.0,0.0
1147,229,3,1072,259,0,0.0,0.0
1148,229,4,1072,259,0,0.0,0.0
1149,229,5,1072,259,0,0.0,0.0
1150,229,6,1072,259,0,0.0,0.0
1151,229,7,1072,259,0,0.0,0.0
1152,229,8,1072,259,0,0.0,0.0
1153,229,9,1072,259,0,0.0,0.0


In [6]:
#プラスとマイナスの反転判定をする関数
def func_fluctuation(x):
    if x.POWER_DIFF * x.PREV_POWER_DIFF < 0:
        return 1
    else:
        return 0
    
#プラスとマイナスの反転判定をする関数を各行から呼び出し
df['FLUCTUATION'] = df.apply(lambda x:func_fluctuation(x),axis=1)

df.query('M_CD == "229"').tail(100)

,M_CD,UP_TIME,POWER,TEMP,ERR_CD,POWER_DIFF,PREV_POWER_DIFF,FLUCTUATION
1199,229,55,1071,270,202,-4.0,-3.0,0
1200,229,56,1071,270,202,0.0,-4.0,0
1201,229,57,1074,270,202,3.0,0.0,0
1202,229,58,1070,270,202,-4.0,3.0,1
1203,229,59,1066,270,202,-4.0,-4.0,0
1204,229,60,1063,270,202,-3.0,-4.0,0
1205,229,61,1068,271,202,5.0,-3.0,1
1206,229,62,1067,272,202,-1.0,5.0,1
1207,229,63,1065,272,202,-2.0,-1.0,0
1208,229,64,1066,272,202,1.0,-2.0,1


pandasで条件分岐(case when的な)によるデータ加工を網羅したい - Qiita<BR>
https://qiita.com/Hyperion13fleet/items/98c31744e66ac1fc1e9f

# ③ カウント型:ジグザグを起こした累積回数
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/f34a53c1-f2ae-cffa-32e9-4f38d05ccd92.png)

In [7]:
#ジグザグを起こした累積回数
df['FLUC_COUNT'] = df.groupby(['M_CD'])['FLUCTUATION'].cumsum()
df.query('M_CD == "229"').tail(100)

,M_CD,UP_TIME,POWER,TEMP,ERR_CD,POWER_DIFF,PREV_POWER_DIFF,FLUCTUATION,FLUC_COUNT
1199,229,55,1071,270,202,-4.0,-3.0,0,2
1200,229,56,1071,270,202,0.0,-4.0,0,2
1201,229,57,1074,270,202,3.0,0.0,0,2
1202,229,58,1070,270,202,-4.0,3.0,1,3
1203,229,59,1066,270,202,-4.0,-4.0,0,3
1204,229,60,1063,270,202,-3.0,-4.0,0,3
1205,229,61,1068,271,202,5.0,-3.0,1,4
1206,229,62,1067,272,202,-1.0,5.0,1,5
1207,229,63,1065,272,202,-2.0,-1.0,0,5
1208,229,64,1066,272,202,1.0,-2.0,1,6


pandasで累積和・累積積（cumsum, cumprod, cummax, cummin） | note.nkmk.me<BR>
https://note.nkmk.me/python-pandas-cumsum-cumprod/

# ④ ステート型：ジグザグが頻繁に起きているか、起きていないかの状態
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/d1ff7905-50ca-a888-eeb0-e0223c1bc679.png)

In [8]:
#先頭行は安定の初期値
df.at[0, 'UNSTABILITY'] = 0
stable_seq_count = 0

#2行目(index=1)からループ処理
for index in range(1,len(df)):
    #既定は直前のステータスを保持する
    df.at[index, 'UNSTABILITY'] = df.at[index-1, 'UNSTABILITY']
    
    #変動があった場合
    if df.at[index, 'FLUCTUATION'] == 1 :
        #連続安定カウントを初期化
        stable_seq_count = 0
        #変動が2回続いた場合、不安定状態判定
        if df.at[index-1, 'FLUCTUATION'] == 1:
            df.at[index, 'UNSTABILITY'] = 1
    #変動がなかった場合、連続安定カウントをアップ
    elif df.at[index, 'FLUCTUATION'] == 0:
        stable_seq_count += 1
    
    #連続安定カウントが5回以上続いた場合かマシンが別のマシンになった場合、安定状態判定
    if stable_seq_count >= 5 or df.at[index, 'M_CD'] != df.at[index-1, 'M_CD']:
        df.at[index, 'UNSTABILITY'] = 0


df.query('M_CD == "229"').head(200)

,M_CD,UP_TIME,POWER,TEMP,ERR_CD,POWER_DIFF,PREV_POWER_DIFF,FLUCTUATION,FLUC_COUNT,UNSTABILITY
1144,229,0,1072,259,0,NaN,NaN,0,0,0.0
1145,229,1,1072,259,0,0.0,NaN,0,0,0.0
1146,229,2,1072,259,0,0.0,0.0,0,0,0.0
1147,229,3,1072,259,0,0.0,0.0,0,0,0.0
1148,229,4,1072,259,0,0.0,0.0,0,0,0.0
1149,229,5,1072,259,0,0.0,0.0,0,0,0.0
1150,229,6,1072,259,0,0.0,0.0,0,0,0.0
1151,229,7,1072,259,0,0.0,0.0,0,0,0.0
1152,229,8,1072,259,0,0.0,0.0,0,0,0.0
1153,229,9,1072,259,0,0.0,0.0,0,0,0.0


Python 3.x - データフレームの中の一個前の列を参照して、次の列の演算を行う　　Python,Pandas｜teratail<BR>
https://teratail.com/questions/64620
<BR>Pythonのfor文によるループ処理（range, enumerate, zipなど） | note.nkmk.me<BR>
https://note.nkmk.me/python-for-usage/

In [9]:
import sys

print(sys.version)

3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) 
[GCC 7.3.0]


In [10]:
print(pd.__version__)

0.24.1
